In [16]:
import sys
!{sys.executable} -m pip install transformers==4.1.1 --user

In [13]:
!pip install transformers==4.1.1
!pip install sentencepiece==0.1.94
!pip install mosestokenizer==1.1.0

^C


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simpletransformers 0.63.9 requires transformers>=4.6.0, but you have transformers 4.1.1 which is incompatible.


  Using cached transformers-4.1.1-py3-none-any.whl (1.5 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.1
    Uninstalling transformers-4.27.1:
      Successfully uninstalled transformers-4.27.1


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-py3-none-any.whl size=49119 sha256=d68063149e2017a3a0309fa5af8ed1b77c2a3978082e123197e2384def63ebbc
  Stored in directory: c:\users\aruna\appdata\local\pip\cache\wheels\79\0d\d1\36adc4fa4953636f9f304c82add5d25be321ff21365d8e47dc
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a6036f44c12358e903ec18d387240f72bb5c8dcb3fe4a6de4f35a55ff610ccb7
  Stored in directory: c:\users\aruna\appdata\local\pip\cache\wheels\ca\cc\e3\f1e272f628fdb013d969acc99cfe2e031ea15b3efb74ffe842
  Created whee

In [1]:
# Imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection

In [6]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# # Library Imports:
# import numpy as np
# import pandas as pd
# import statistics as st
# from sklearn import feature_extraction, linear_model, model_selection, preprocessing
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# import seaborn as sns
# import regex
# import gc
# import re
# import string
# import operator
# from collections import defaultdict
# import sys
# import tokenization
# from wordcloud import STOPWORDS
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler

# #  Metrics Import:
# from sklearn.metrics import classification_report 
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import confusion_matrix
# from sklearn import metrics

In [2]:
# Reading in Datasets:
df_train_cleanedMislabelsDuplicates = pd.read_csv("../Data/df_train_cleanedMislabelsDuplicates.csv", encoding='utf-8')
df_train_cleanedNoMislabelsDuplicates = pd.read_csv("../Data/df_train_cleanedNoMislabelsDuplicates.csv", encoding='utf-8')
df_train_cleanedMislabelsNoDuplicates = pd.read_csv("../Data/df_train_cleanedMislabelsNoDuplicates.csv", encoding='utf-8')
df_train_cleanedNoMislabelsNoDuplicates = pd.read_csv("../Data/df_train_cleanedNoMislabelsNoDuplicates.csv", encoding='utf-8')
df_test_cleaned = pd.read_csv("../Data/df_test_cleaned.csv", encoding='utf-8')

In [3]:
df_train_cleanedNoMislabelsNoDuplicates

,id,keyword,location,text,target_relabeled
0,1,no_keyword,no_location,our deeds are the reason of this earthquake ma...,1
1,4,no_keyword,no_location,forest fire near los angeles ronge sask canada,1
2,5,no_keyword,no_location,all residents asked to shelter in place are be...,1
3,6,no_keyword,no_location,13000 people receive wildfires evacuation orde...,1
4,7,no_keyword,no_location,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
6882,10859,no_keyword,no_location,breaking los angeles refugio oil spill may hav...,1
6883,10860,no_keyword,no_location,a siren just went off and it was not the forne...,1
6884,10862,no_keyword,no_location,officials say a quarantine is in place at an a...,1
6885,10864,no_keyword,no_location,on the flip side i am at walmart and there is ...,1


In [3]:
dfs = [df_train_cleanedMislabelsDuplicates, df_train_cleanedNoMislabelsDuplicates, df_train_cleanedMislabelsNoDuplicates, df_train_cleanedNoMislabelsNoDuplicates]

In [4]:
df_train_cleanedMislabelsNoDuplicates.drop(['location', 'id'], axis=1, inplace=True)

In [ ]:
# try changing tokenizer_class to processor_class or decreasing python version

In [20]:
from transformers import MarianMTModel, MarianTokenizer
target_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name).to('cuda:0')

RuntimeError: Failed to import transformers.models.marian.modeling_marian because of the following error (look up to see its traceback):
add_code_sample_docstrings() got an unexpected keyword argument 'tokenizer_class'

In [ ]:
en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name).to('cuda:0')

In [21]:
def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts,return_tensors="pt").to('cuda:0')
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

In [22]:
def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    fr_texts = translate(texts, target_model, target_tokenizer, 
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, 
                                      language=source_lang)
    
    return back_translated_texts

In [23]:
en_texts = ['This is so cool', 'I hated the food', 'They were very helpful']
aug_texts = back_translate(en_texts, source_lang="en", target_lang="es")
print(aug_texts)

NameError: name 'target_model' is not defined

In [ ]:
translated_text = back_translate(list(df['text']), source_lang="en", target_lang="fr")
print(df.head())

In [25]:
#specify simple transformer model with number of epochs
model_args = ClassificationArgs(num_train_epochs = 4,learning_rate = 3e-6)
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args, use_cuda=False)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
#Train model and evaluate test set
model.train_model(df_train_cleanedMislabelsNoDuplicates)

C:\Users\aruna\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/6887 [00:00<?, ?it/s]

In [ ]:
# Getting Test DataFrame ready to be used for predictions:
test.drop(['keyword', 'location', 'id'], axis=1, inplace=True)
test.info()

In [ ]:
# Outputing Predictions:
predictions, raw_outputs = model.predict(list(test['text']))
print(predictions)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = predictions
submission.to_csv('submission.csv')


## 2nd Model Attempt:

In [7]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
# import urllib.request
# url = 'https://github.com/tensorflow/models/blob/master/official/nlp/tools/tokenization.py'
# filename = 'tokenization.py'
# urllib.request.urlretrieve(url, filename)

('tokenization.py', <http.client.HTTPMessage at 0x28659a47040>)

In [5]:
# import gc
# import re
# import string
# import operator
# from collections import defaultdict


# import numpy as np
# import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# import matplotlib.pyplot as plt
# import seaborn as sns

import tokenization
# from wordcloud import STOPWORDS

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

SEED = 1337

In [6]:
K = 2
skf = StratifiedKFold(n_splits=K, random_state=SEED, shuffle=True)

DISASTER = df_train_cleanedMislabelsNoDuplicates['target'] == 1
print('Whole Training Set Shape = {}'.format(df_train_cleanedMislabelsNoDuplicates.shape))
print('Whole Training Set Unique keyword Count = {}'.format(df_train_cleanedMislabelsNoDuplicates['keyword'].nunique()))
print('Whole Training Set Target Rate (Disaster) {}/{} (Not Disaster)'.format(df_train_cleanedMislabelsNoDuplicates[DISASTER]['target'].count(), df_train_cleanedMislabelsNoDuplicates[~DISASTER]['target'].count()))

for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train_cleanedMislabelsNoDuplicates['text'], df_train_cleanedMislabelsNoDuplicates['target']), 1):
    print('\nFold {} Training Set Shape = {} - Validation Set Shape = {}'.format(fold, df_train_cleanedMislabelsNoDuplicates.loc[trn_idx, 'text'].shape, df_train_cleanedMislabelsNoDuplicates.loc[val_idx, 'text'].shape))
    print('Fold {} Training Set Unique keyword Count = {} - Validation Set Unique keyword Count = {}'.format(fold, df_train_cleanedMislabelsNoDuplicates.loc[trn_idx, 'keyword'].nunique(), df_train_cleanedMislabelsNoDuplicates.loc[val_idx, 'keyword'].nunique()))   

Whole Training Set Shape = (6887, 3)
Whole Training Set Unique keyword Count = 222
Whole Training Set Target Rate (Disaster) 2812/4075 (Not Disaster)

Fold 1 Training Set Shape = (3443,) - Validation Set Shape = (3444,)
Fold 1 Training Set Unique keyword Count = 222 - Validation Set Unique keyword Count = 222

Fold 2 Training Set Shape = (3444,) - Validation Set Shape = (3443,)
Fold 2 Training Set Unique keyword Count = 222 - Validation Set Unique keyword Count = 222


In [7]:
class ClassificationReport(Callback):
    
    def __init__(self, train_data=(), validation_data=()):
        super(Callback, self).__init__()
        
        self.X_train, self.y_train = train_data
        self.train_precision_scores = []
        self.train_recall_scores = []
        self.train_f1_scores = []
        
        self.X_val, self.y_val = validation_data
        self.val_precision_scores = []
        self.val_recall_scores = []
        self.val_f1_scores = [] 
               
    def on_epoch_end(self, epoch, logs={}):
        train_predictions = np.round(self.model.predict(self.X_train, verbose=0))        
        train_precision = precision_score(self.y_train, train_predictions, average='macro')
        train_recall = recall_score(self.y_train, train_predictions, average='macro')
        train_f1 = f1_score(self.y_train, train_predictions, average='macro')
        self.train_precision_scores.append(train_precision)        
        self.train_recall_scores.append(train_recall)
        self.train_f1_scores.append(train_f1)
        
        val_predictions = np.round(self.model.predict(self.X_val, verbose=0))
        val_precision = precision_score(self.y_val, val_predictions, average='macro')
        val_recall = recall_score(self.y_val, val_predictions, average='macro')
        val_f1 = f1_score(self.y_val, val_predictions, average='macro')
        self.val_precision_scores.append(val_precision)        
        self.val_recall_scores.append(val_recall)        
        self.val_f1_scores.append(val_f1)
        
        print('\nEpoch: {} - Training Precision: {:.6} - Training Recall: {:.6} - Training F1: {:.6}'.format(epoch + 1, train_precision, train_recall, train_f1))
        print('Epoch: {} - Validation Precision: {:.6} - Validation Recall: {:.6} - Validation F1: {:.6}'.format(epoch + 1, val_precision, val_recall, val_f1))  

In [8]:
%%time

bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1', trainable=True)

CPU times: total: 6.16 s
Wall time: 14.7 s


In [9]:
class DisasterDetector:
    
    def __init__(self, bert_layer, max_seq_length=128, lr=0.0001, epochs=15, batch_size=32):
        
        # BERT and Tokenization params
        self.bert_layer = bert_layer
        
        self.max_seq_length = max_seq_length        
        vocab_file = self.bert_layer.resolved_object.vocab_file.asset_path.numpy()
        do_lower_case = self.bert_layer.resolved_object.do_lower_case.numpy()
        self.tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
        
        # Learning control params
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        
        self.models = []
        self.scores = {}
        
    def encode(self, texts):
                
        all_tokens = []
        all_masks = []
        all_segments = []

        for text in texts:
            text = self.tokenizer.tokenize(text)
            text = text[:self.max_seq_length - 2]
            input_sequence = ['[CLS]'] + text + ['[SEP]']
            pad_len = self.max_seq_length - len(input_sequence)

            tokens = self.tokenizer.convert_tokens_to_ids(input_sequence)
            tokens += [0] * pad_len
            pad_masks = [1] * len(input_sequence) + [0] * pad_len
            segment_ids = [0] * self.max_seq_length

            all_tokens.append(tokens)
            all_masks.append(pad_masks)
            all_segments.append(segment_ids)

        return np.array(all_tokens), np.array(all_masks), np.array(all_segments)
    def build_model(self):
        
        input_word_ids = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_word_ids')
        input_mask = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_mask')
        segment_ids = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='segment_ids')    
        
        pooled_output, sequence_output = self.bert_layer([input_word_ids, input_mask, segment_ids])   
        clf_output = sequence_output[:, 0, :]
        out = Dense(1, activation='sigmoid')(clf_output)
        
        model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
        optimizer = SGD(learning_rate=self.lr, momentum=0.8)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        return model
    
    def train(self, X):

        for fold, (trn_idx, val_idx) in enumerate(skf.split(X['text'], X['keyword'])):
            print('\nFold {}\n'.format(fold))
            X_trn_encoded = self.encode(X.loc[trn_idx, 'text'].str.lower())
            y_trn = X.loc[trn_idx, 'target']
            X_val_encoded = self.encode(X.loc[val_idx, 'text'].str.lower())
            y_val = X.loc[val_idx, 'target']
        
            # Callbacks
            metrics = ClassificationReport(train_data=(X_trn_encoded, y_trn), validation_data=(X_val_encoded, y_val))
            
            # Model
            model = self.build_model()        
            model.fit(X_trn_encoded, y_trn, validation_data=(X_val_encoded, y_val), callbacks=[metrics], epochs=self.epochs, batch_size=self.batch_size)
            
            self.models.append(model)
            self.scores[fold] = {
                'train': {
                    'precision': metrics.train_precision_scores,
                    'recall': metrics.train_recall_scores,
                    'f1': metrics.train_f1_scores                    
                },
                'validation': {
                    'precision': metrics.val_precision_scores,
                    'recall': metrics.val_recall_scores,
                    'f1': metrics.val_f1_scores                    
                }
            }
                    
                
    def plot_learning_curve(self):
        
        fig, axes = plt.subplots(nrows=K, ncols=2, figsize=(20, K * 6), dpi=100)
    
        for i in range(K):
            
            # Classification Report curve
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[i].history.history['val_accuracy'], ax=axes[i][0], label='val_accuracy')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['precision'], ax=axes[i][0], label='val_precision')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['recall'], ax=axes[i][0], label='val_recall')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['f1'], ax=axes[i][0], label='val_f1')        

            axes[i][0].legend() 
            axes[i][0].set_title('Fold {} Validation Classification Report'.format(i), fontsize=14)

            # Loss curve
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['loss'], ax=axes[i][1], label='train_loss')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['val_loss'], ax=axes[i][1], label='val_loss')

            axes[i][1].legend() 
            axes[i][1].set_title('Fold {} Train / Validation Loss'.format(i), fontsize=14)

            for j in range(2):
                axes[i][j].set_xlabel('Epoch', size=12)
                axes[i][j].tick_params(axis='x', labelsize=12)
                axes[i][j].tick_params(axis='y', labelsize=12)

        plt.show()
        
        
    def predict(self, X):
        
        X_test_encoded = self.encode(X['text'].str.lower())
        y_pred = np.zeros((X_test_encoded[0].shape[0], 1))
        for model in self.models:
            y_pred += model.predict(X_test_encoded) / len(self.models)

        return y_pred

In [ ]:
# Training, Evaluation, and Prediction:
clf = DisasterDetector(bert_layer, max_seq_length=128, lr=0.0001, epochs=10, batch_size=32)
clf.train(df_train_cleanedMislabelsNoDuplicates)


Fold 0

Epoch 1/10
108/108 [==============================] - ETA: 0s - loss: 0.6337 - accuracy: 0.6520 
Epoch: 1 - Training Precision: 0.735271 - Training Recall: 0.696455 - Training F1: 0.701831
Epoch: 1 - Validation Precision: 0.719405 - Validation Recall: 0.683083 - Validation F1: 0.686151
108/108 [==============================] - 9695s 90s/step - loss: 0.6337 - accuracy: 0.6520 - val_loss: 0.5972 - val_accuracy: 0.7152
Epoch 2/10
108/108 [==============================] - ETA: 0s - loss: 0.5509 - accuracy: 0.7557 
Epoch: 2 - Training Precision: 0.772535 - Training Recall: 0.747783 - Training F1: 0.754368
Epoch: 2 - Validation Precision: 0.762338 - Validation Recall: 0.741017 - Validation F1: 0.746263
108/108 [==============================] - 11459s 107s/step - loss: 0.5509 - accuracy: 0.7557 - val_loss: 0.5257 - val_accuracy: 0.7622
Epoch 3/10
108/108 [==============================] - ETA: 0s - loss: 0.4948 - accuracy: 0.7766 
Epoch: 3 - Training Precision: 0.786197 - Training

In [ ]:
clf.plot_learning_curve()

In [ ]:
y_pred = clf.predict(df_test)

model_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
model_submission['target'] = np.round(y_pred).astype('int')
model_submission.to_csv('model_submission.csv', index=False)
model_submission.describe()

#### Building Vectors:
We use sckit-learn's CountVectorizer to count the words in each tweet and turn them into data our machine learning model can process

In [16]:
count_vectorizer = feature_extraction.text.CountVectorizer()

train_vectors = count_vectorizer.fit_transform(df_train_cleanedNoMislabelsNoDuplicates['text'])

#For test_vectors, we only use .transform() function b/c we need that the train and test vectors use the same set of tokens
test_vectors = count_vectorizer.transform(df_test_cleaned["text"])

#### Building Model:
We use the words contained in each tweet as the predictor variable for real/fake disaster tweet (1/0). We assume a linear model/connection in this tutorial

In [17]:
# Since the vectors are big, we want to push the model's weights toward 0 without completely discounting different words
# Thus, we use Ridge Regression
clf = linear_model.RidgeClassifier()
# cv = 3 means we are using Three-Fold Cross Validation
scores = model_selection.cross_val_score(clf, train_vectors, df_train_cleanedNoMislabelsNoDuplicates['target_relabeled'], cv = 3, scoring = "f1")
print("Scores: ", scores)

Scores:  [0.5767098  0.53946621 0.5949214 ]


Ways to improve this score: Do TFIDF, LSA, LSTM/RNNs, etc

#### Predictions and Submission File:

In [10]:
clf.fit(train_vectors, train_df['target'])
tutorial_submission = pd.read_csv("../Data/sample_submission.csv")
tutorial_submission['target'] = clf.predict(test_vectors)
tutorial_submission.to_csv("../Result Files/submission.csv", index = False)